In [ ]:
import numpy as np

class NEB:
    def __init__(self, energy_matrix, minima, num_images=10, k_spring=1.0, max_iter=1000, tol=1e-5):
        """
        energy_matrix: 2D numpy array where each element is z = f(x,y)
        minima: List of (x, y) tuples representing local minima
        num_images: Number of intermediate images (discretization of the path)
        k_spring: Spring constant for the elastic band
        max_iter: Maximum number of iterations
        tol: Convergence tolerance
        """
        self.energy_matrix = energy_matrix
        self.shape = energy_matrix.shape
        self.minima = minima
        self.num_images = num_images
        self.k_spring = k_spring
        self.max_iter = max_iter
        self.tol = tol

    def get_energy(self, x, y):
        """Handles wrap-around (periodic boundary conditions)"""
        x = x % self.shape[0]
        y = y % self.shape[1]
        return self.energy_matrix[x, y]

    def get_gradient(self, x, y):
        """Computes finite-difference gradient with periodic boundaries."""
        x_p, x_m = (x + 1) % self.shape[0], (x - 1) % self.shape[0]
        y_p, y_m = (y + 1) % self.shape[1], (y - 1) % self.shape[1]

        grad_x = (self.get_energy(x_p, y) - self.get_energy(x_m, y)) / 2.0
        grad_y = (self.get_energy(x, y_p) - self.get_energy(x, y_m)) / 2.0
        return np.array([grad_x, grad_y])

    def initialize_band(self, min1, min2):
        """Creates a linear interpolation between two minima"""
        x1, y1 = min1
        x2, y2 = min2
        return np.linspace([x1, y1], [x2, y2], self.num_images)

    def optimize_path(self, path):
        """Optimizes the NEB path using gradient descent"""
        for _ in range(self.max_iter):
            new_path = path.copy()
            max_displacement = 0

            for i in range(1, self.num_images - 1):  # Ignore endpoints
                x, y = path[i]
                grad = self.get_gradient(int(x), int(y))

                # Compute spring force
                tangent = path[i + 1] - path[i - 1]
                tangent /= np.linalg.norm(tangent) + 1e-8  # Normalize
                spring_force = self.k_spring * (np.linalg.norm(path[i + 1] - path[i]) - np.linalg.norm(path[i] - path[i - 1])) * tangent

                # Compute perpendicular force
                total_force = -grad + spring_force
                new_path[i] += 0.1 * total_force  # Small update step

                max_displacement = max(max_displacement, np.linalg.norm(total_force))

            path = new_path
            if max_displacement < self.tol:
                break

        return path

    def find_mep(self):
        """Finds the minimum energy paths between all local minima"""
        paths = {}
        for i in range(len(self.minima)):
            for j in range(i + 1, len(self.minima)):
                min1, min2 = self.minima[i], self.minima[j]
                path = self.initialize_band(min1, min2)
                optimized_path = self.optimize_path(path)
                paths[(min1, min2)] = optimized_path
        return paths


# Example Usage
energy_matrix = np.random.rand(20, 20)  # Replace with actual function values
local_minima = [(5, 5), (15, 15)]  # Example minima positions

neb_solver = NEB(energy_matrix, local_minima)
mep_paths = neb_solver.find_mep()

# Print paths
for (min1, min2), path in mep_paths.items():
    print(f"Path from {min1} to {min2}:")
    print(path)
